# Segunda entrega - 51940 Cueva Tomas
En este notebook se desarrolla un proceso ETL. El mismo consiste de la extracción de datos de la API de CoinGecko, su transformación y filtrado a través de la biblioteca Pandas, finalizando con la carga de los datos utilizando PySpark.

## Instalo bibliotecas y llamo a los modulos del paquete etl de este directorio

In [1]:
!pip install requests SQLAlchemy psycopg2-binary pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 875.8 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 800.4 kB/s eta 0:00:00a 0:00:01


In [6]:
from etl import(
    spark,
    extract_CoinGecko_API as extract,
    transform_CoinGecko_API as transform,
    load_pyspark_redshift_connectr as load
)

## Inicializo las clases de los modulo.

In [10]:
extr = extract.Extract("https://api.coingecko.com/api/v3/")
spark_session = spark.PySpark()

## 100 criptos con mayor capitalización bursatil
En las siguientes celdas se extraen los datos de las 100 criptomonedas con mayor capitalizazción bursatil, se convierten en un dataframe y se cargan a su tabla correspondiente de redshift,

In [11]:
json_top = extr.get_criptos_top()

In [15]:
df = spark_session.spark.read.json(
            spark_session.spark.sparkContext.parallelize(json_top), multiLine=True
        )

In [19]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- symbol: string (nullable = true)
 |-- name: string (nullable = true)
 |-- current_price: double (nullable = true)
 |-- market_cap: long (nullable = true)
 |-- market_cap_rank: long (nullable = true)
 |-- total_volume: long (nullable = true)
 |-- high_24h: double (nullable = true)
 |-- low_24h: double (nullable = true)
 |-- price_change_24h: double (nullable = true)
 |-- price_change_percentage_24h: double (nullable = true)
 |-- market_cap_change_24h: double (nullable = true)
 |-- market_cap_change_percentage_24h: double (nullable = true)
 |-- circulating_supply: double (nullable = true)
 |-- ath: double (nullable = true)
 |-- ath_change_percentage: double (nullable = true)
 |-- ath_date: string (nullable = true)
 |-- atl: double (nullable = true)
 |-- atl_change_percentage: double (nullable = true)
 |-- atl_date: string (nullable = true)
 |-- last_updated: string (nullable = true)



In [18]:
# Seleccionar las columnas deseadas
selected_columns = [
    'id',
    'symbol',
    'name',
    'current_price',
    'market_cap',
    'market_cap_rank',
    'total_volume',
    'high_24h',
    'low_24h',
    'price_change_24h',
    'price_change_percentage_24h',
    'market_cap_change_24h',
    'market_cap_change_percentage_24h',
    'circulating_supply',
    'ath',
    'ath_change_percentage',
    'ath_date',
    'atl',
    'atl_change_percentage',
    'atl_date',
    'last_updated',
]
df = df.select(selected_columns)

In [21]:
table = "criptos_market_cap"
df.write \
    .format("jdbc") \
    .option("url", self.REDSHIFT_URL) \
    .option("dbtable", table) \
    .option("user", self.REDSHIFT_USER) \
    .option("password", self.REDSHIFT_PASSWORD) \
    .option("driver", "org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

print("Dataframe subido")

NameError: name 'self' is not defined

### Market chart de las top 10
Utilizando los datos del dataframe del top 100, vamos a obtener el market chart de las 10 primeras del top.

In [7]:
id_column = df_top['id']

In [8]:
id_array = id_column.to_numpy()

Luego ded obtener el array con todos los id, vamos a iterar 10 veces sobre el contenido del array, vamos a extraer los datos y transformarlos con las funciones de los modulos "extr" y "transform", para finalmente concatenarlos en un unico DataFrame.

In [9]:
df_all_market_charts = pd.DataFrame(columns = ['timestamp', 'prices', 'market_caps', 'total_volumes', 'cripto'])

for i in range(0,10):
    data = extr.get_market_chart(id_array[i])
    df = transform.json_to_df_market_chart(data,id_array[i])
    df_all_market_charts = pd.concat([df_all_market_charts, df], ignore_index=True)

Solicitud exitosa
Solicitud exitosa
Solicitud exitosa
Solicitud exitosa
Solicitud exitosa
Solicitud exitosa
Solicitud exitosa
Solicitud exitosa
Solicitud exitosa
Solicitud exitosa


In [10]:
df_all_market_charts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25613 entries, 0 to 25612
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   timestamp      25613 non-null  object 
 1   prices         25613 non-null  float64
 2   market_caps    25610 non-null  float64
 3   total_volumes  25613 non-null  float64
 4   cripto         25613 non-null  object 
dtypes: float64(3), object(2)
memory usage: 1000.6+ KB


In [11]:
table = "market_chart_criptos"
ld.execute(df_all_market_charts, table)

Ejecutando ETL
Convertir el DataFrame de pandas a un PySpark DataFrame
DataFrame[timestamp: bigint, prices: double, market_caps: double, total_volumes: double, cripto: string]
Cargar el PySpark DataFrame en Redshift
Dataframe subido
